# 23 - Challenges with Effect Heterogeneity and Nonlinearity

개별 단위 수준에서 처치 효과를 예측하는 것은 근거가 부족하여 매우 어렵습니다. 우리가 단 하나의 잠재적 결과 $T(t)$ 만 관찰할 수 있으므로, 이를 직접 추정할 수는 없습니다. 대신, 우리는 목표 변환(영리하게 설계된 손실 함수와 같이)에 의존하여 조건부 처치 효과를 기대치로만 추정해야 합니다. 그러나 이것이 유일한 과제는 아닙니다. 처치 효과가 매우 불확실하고 예측하기 어렵기 때문에, 그것을 추정하는 값들은 종종 상당한 변동성을 가지고 있습니다. 이것은 개인화된 처치 할당과 같이 처치 효과에 따라 단위를 세분화하고자 하는 응용 프로그램에 큰 실질적인 영향을 미칩니다.

우리는 때때로, 조건부 평균 처치 효과(CATE)를 직접 추정하기보다는 다른 대체 목표에 집중함으로써 더 나은 처치 효과 세분화를 얻을 수 있음을 볼 것입니다. 이 대체 목표는 일반적으로 더 적은 변동성을 가집니다. 이것이 일반적으로 발생하는 경우는 관심 있는 결과 변수$Y$가 이진(binary)일 때입니다.

## Treatment Effects on Binary Outcomes




In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.formula.api as smf
from matplotlib import pyplot as plt
from matplotlib import style
style.use("ggplot")

C:\Users\user\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


AttributeError: module 'numpy' has no attribute 'MachAr'

In [ ]:
from typing import List

import numpy as np
import pandas as pd
from toolz import curry, partial

@curry
def avg_treatment_effect(df, treatment, outcome):
    return df.loc[df[treatment] == 1][outcome].mean() - df.loc[df[treatment] == 0][outcome].mean()
    
    

@curry
def cumulative_effect_curve(df: pd.DataFrame,
                            treatment: str,
                            outcome: str,
                            prediction: str,
                            min_rows: int = 30,
                            steps: int = 100,
                            effect_fn = avg_treatment_effect) -> np.ndarray:
    
    size = df.shape[0]
    ordered_df = df.sort_values(prediction, ascending=False).reset_index(drop=True)
    n_rows = list(range(min_rows, size, size // steps)) + [size]
    return np.array([effect_fn(ordered_df.head(rows), treatment, outcome) for rows in n_rows])


@curry
def cumulative_gain_curve(df: pd.DataFrame,
                          treatment: str,
                          outcome: str,
                          prediction: str,
                          min_rows: int = 30,
                          steps: int = 100,
                          effect_fn = avg_treatment_effect) -> np.ndarray:
    

    size = df.shape[0]
    n_rows = list(range(min_rows, size, size // steps)) + [size]

    cum_effect = cumulative_effect_curve(df=df, treatment=treatment, outcome=outcome, prediction=prediction,
                                         min_rows=min_rows, steps=steps, effect_fn=effect_fn)

    return np.array([effect * (rows / size) for rows, effect in zip(n_rows, cum_effect)])

테크 회사에서 일하게 되면 맞닥뜨릴 수 있는 매우 흔한 문제는 다음과 같습니다: 경영진은 어떤 형태의 유도 수단을 통해 제품으로의 고객 전환을 증가시키고자 합니다. 예를 들어, 인앱 구매를 위한 10 BRL 바우처를 제공하여 앱 설치 수를 늘리거나, 첫 번째 탑승 시 무료 탑승을 제공하거나, 투자 플랫폼에서 처음 세 달 동안 거래 수수료를 낮추는 등의 조치를 취할 수 있습니다. 이러한 유도 수단은 종종 비용이 많이 들기 때문에, 모든 사람에게 적용하기보다는 특히 그 효과에 민감한 고객들에게만 적용하는 것이 이상적입니다.

인과 추론 용어로 볼 때, 이러한 유형의 비즈니스 문제는 처치 효과의 이질성(Treatment Effect Heterogeneity, TEH)에 속한다고 할 수 있습니다. 구체적으로, 비용이 드는 유도 수단을 처치 $T$로, 전환을 이진 결과 $Y$로, 고객별 처치 전 특성 $X$을 고려합니다. 그런 다음, Double/Debiased ML과 같은 방법으로 조건부 평균 처치 효과 $E[Y_{1}-Y_{0}|X]$ (또는 $E[Y^{'}(T)|X]$ )를 추정하고, 추정된 처치 효과가 가장 높은 고객들에게만 유도 수단을 적용할 수 있습니다. 비즈니스 용어로는, 고객 전환 전략을 개인화하는 것입니다. 높은 전환 증가율을 보이는 고객 세그먼트를 찾아 그들에게만 유도 수단을 사용하는 것입니다.

그러나, 결과 값이 이진(binary)라는 것은 TEH 접근법이 명확하지 않다는 문제를 발생시킵니다. 이 내용은 직관에 반하는 것이기 때문에, 먼저 무슨 일이 일어나는지 보여주고 그 후에 이유를 설명하겠습니다.

## Simulating Some Data

간단하지만 관련성 있게 생각해봅시다. 우리는 처치인 유도 수단(넛지)을 완전 무작위인 베르누이 분포에서 $p=0.5$를 가지고 추출된 것으로 시뮬레이션할 것입니다. 이것은 처치가 동전전 던지기와 같다는 것을 의미합니다. 또한 이것은 우리가 주의해야 할 혼동 요인이 없다는 것을 의미하기도 합니다.

$ nudge \sim \mathcal{B}(0.5) $

다음으로, 고객의 공변량인 연령과 소득을 감마 분포를 따라 시뮬레이션하겠습니다. 이는 고객에 대해 이미 알고 있는 정보이며, 이를 바탕으로 개인화하고자 합니다. 즉, 연령과 소득에 따라 정의된 고객 그룹 중 유도 처치에 매우 민감하게 반응하는 그룹을 찾고자 하는 것입니다.
 
$ age \sim G(10, 4) $
 
$ income \sim G(20, 2) $
 
마지막으로, 우리는 고객의 전환을 시뮬레이션할 것입니다. 이를 위해, 먼저 랜덤 노이즈가 존재하는 **선형 모델의 잠재 변수를 생성합니다.** 중요한 것은 `소득`이 $Y_{latent}$의 예측에 매우 중요하지만, **처치 효과에 영향을 주지 않는다는 것**입니다. 간단히 말해서, 유도는 모든 `소득 수준`에 대해 $Y_{latent}$를 같게 증가시킵니다. 반대로, `연령`은 `유도` 처치와의 상호작용을 통해서만 $Y_{latent}$ 에 영향을 미칩니다.

$Y_{latent} \sim N(-4.5 + 0.001 \ income + nudge + 0.01 \ nudge \ age, 1)$

$Y_{latent}$ 를 구한다면, 이를 $Y_{latent}>x$인 경우로 설정하여 `전환`을 시뮬레이션할 수 있습니다. 먼저, `x=0`으로 설정하여 전환율이 대략 50\가 되도록 합니다. 즉, 평균적으로 고객의 50\%가 제품으로 전환한다는 의미입니다.
 
$conversion = 1\{Y_{latent} > 0\}$

In [2]:
np.random.seed(123)

n = 100000
nudge = np.random.binomial(1, 0.5, n)
age = np.random.gamma(10, 4, n)
estimated_income = np.random.gamma(20, 2, n)*100

latent_outcome = np.random.normal(-4.5 + estimated_income*0.001 + nudge + nudge*age*0.01)
conversion = (latent_outcome > .1).astype(int)

NameError: name 'np' is not defined

편의를 위해 모든 것을 데이터프레임에 저장합시다. 또한, 실제로 평균 전환율이 50%에 근접한지 확인합시다.

In [ ]:
df = pd.DataFrame(dict(conversion=conversion,
                       nudge=nudge,
                       age=age,
                       estimated_income=estimated_income,
                       latent_outcome=latent_outcome))

df.mean()

평균 처치 효과에 대해서, 처치가 무작위화되었기 때문에 우리는 이를 처치 그룹과 대조 그룹 간의 단순한 평균 차이로 추정할 수 있습니다:
$E[Y|T=1] - E[Y|T=0]$. 그러니, 이러한 평균 처치 효과가 어떻게 보이는지 봅시다. 우리는 잠재적 결과와 전환을 모두 살펴볼 것이며 여기 중요한 사실이 있습니다.

In [ ]:
df.groupby("nudge")[["latent_outcome", "conversion"]].mean()

In [ ]:
avg_treatment_effect(df, "nudge", "latent_outcome")

In [ ]:
avg_treatment_effect(df, "nudge", "conversion")

잠재적 결과에 대한 ATE는 비교적 명확합니다. 우리의 데이터 생성 모델에서, 이 효과는 `1 + avg(age)*0.01`이 되어야 합니다. 평균 연령이 약 40이므로, 이것은 ATE를 약 1.4로 만듭니다. 더 흥미롭고 (복잡하기도한) 부분은 바로 전환에 대한 ATE입니다. **전환은 0과 1 사이로 제한되어 있기 때문에, ATE는 선형이 아닙니다.** 따라서, 우리는 잠재적 결과와 같이 간단한 공식으로부터 이를 유추할 수 없습니다(복잡한 공식이 있지만). 효과가 작다고 해봅시다. 이는 합리적일까요 ?  전환율이 100%를 넘을 수 없기 때문에, 처치가 전환율을 1.4포인트 증가시킬 수 없습니다. 다음에 나올 내용은 이해하기 위해서 앞선 내용을 꼭 기억하시기 바랍니다.
 
이제 조건부 평균 처치 효과(CATE)에 대해 이야기합시다. 우리의 데이터 생성 과정을 살펴보면, `추정 소득`이 전환을 예측하지만 전환에 대한 유도의 효과엔 영향을 주지 않는 것을 알 수 있습니다. 그러므로, `추정 소득`에 기반하여 고객을 세분화하면 동일한 처치 효과를 가진 세그먼트가 생성됩니다. 반면, `연령`은 유도와의 상호작용을 통해서만 전환에 영향을 미칩니다. 따라서 다른 연령 세그먼트는 처치에 매우 다르게 반응할 것이며, 다른 소득 세그먼트는 그렇지 않을 것입니다. 즉, `추정 소득`은 좋은 개인화 변수가 아니어야 하며, 연령은 그렇게 되어야 합니다.
 
누적 효과 곡선을 통해 살펴보겠습니다. `연령`에 대한 곡선은 ATE에서 매우 멀리 시작하여 점차 수렴해야 하며, `추정 소득`에 대한 곡선은 ATE 주변에서만 변동해야 합니다. 이것이 바로 누적 효과 곡선을 그렸을때 볼 수 있는 잠재적 결과에 대한 유도의 정확한 효과입니다.

In [ ]:
cumulative_effect_fn = cumulative_effect_curve(df, "nudge", "latent_outcome", min_rows=500)

age_cumm_effect_latent = cumulative_effect_fn(prediction="age")
inc_cumm_effect_latent = cumulative_effect_fn(prediction="estimated_income")

plt.plot(age_cumm_effect_latent, label="age")
plt.plot(inc_cumm_effect_latent, label="est. income")
plt.legend()
plt.xlabel("Percentile")
plt.ylabel("Effect on Latet Outcome");

다시 말하지만, 잠재적 결과는 매우 좋습니다. 그 선형성 때문에 우리의 기대는 현실과 상당히 가깝습니다. 하지만 실제 생활에서 우리는 잠재적 결과를 신경 쓰지 않으며, 가지고 있지도 않습니다. 우리가 가진 것은 전환 뿐입니다. 전환에서 상황은 훨씬 더 복잡해 보입니다. 누적 효과 곡선을 그리면, `연령`은 여전히 일부 처치 효과의 이질성을 보여주며, ATE 위에서 시작하여 점차 그것에 수렴합니다. 이는 연령이 높을수록 처치 효과가 높다는 것을 의미합니다. 지금까지는 좋습니다. 이것은 우리가 기대하는 바입니다.

In [ ]:
cumulative_effect_fn = cumulative_effect_curve(df, "nudge", "conversion", min_rows=500)

age_cumm_effect_latent = cumulative_effect_fn(prediction="age")
inc_cumm_effect_latent = cumulative_effect_fn(prediction="estimated_income")

plt.plot(age_cumm_effect_latent, label="age")
plt.plot(inc_cumm_effect_latent, label="est. income")
plt.legend()
plt.xlabel("Percentile")
plt.ylabel("Effect on Conversino");

그러나, 우리는 `추정 소득`에 의한 아주 많은 처치 효과의 이질성 또한 지니고 있습니다. 높은 `추정 소득`을 가진 고객들은 훨씬 낮은 처치 효과를 보이며, 이로 인해 누적 효과 곡선은 처음에는 제로까지 떨어진 후에 천천히 ATE로 수렴합니다. 개인화 관점에서 볼 때 `추정 소득`은 `연령`을 통해 얻어질 세그먼트와 비교했을때, 더 많은 처치 효과의 이질성(TEH)을 가진 세그먼트를 생성할 것임을 알 수 있습니다.

불편한 사실이죠? 우리가 효과 이질성을 가져온다고 알고 있는 특성인 `연령`이, 처치 효과에 영향을 주지 않는다고 알려진 특성(`추정 소득`)과 비교할 때 개인화에 있어서 더 나쁘다는 것은 어떻게 된 일일까요? 대답은 **결과 함수의 비선형성**에 있습니다. 비록 추정 소득이 잠재적 결과에 대한 유도의 효과를 영향을 주진 않지만, 그 잠재적 결과를 전환으로 변환할 때는 그렇지 않습니다(적어도 간접적으로). 전환은 선형이 아닙니다. 이는 **미분이 위치에 따라 달라진다는 것**을 의미합니다. 전환은 최대 1까지만 올라갈 수 있으므로, 이미 매우 높다면 증가시키기 어렵습니다. 다시 말해, 높은 전환의 미분은 매우 낮습니다. 하지만 전환은 0에서도 제한되므로, 이미 매우 낮다면 미분도 낮을 것입니다. 전환은 양 끝에서 낮은 미분을 가진 S 형태를 따릅니다. 이는 추정된 소득 구간별(100씩 구간) 평균 전환을 그래프로 그려서 확인할 수 있습니다.

In [ ]:
(df
 .assign(estimated_income_bins=(df["estimated_income"]/100).astype(int)*100)
 .groupby("estimated_income_bins")
 [["conversion"]]
 .mean()
 .plot()
);

전환율이 매우 높을 때 이 곡선의 기울기(미분)가 매우 작다는 점을 주목하세요. 또한, 전환율이 매우 낮을 때도 기울기는 작습니다 ( 표본이 적어 쉽게 보긴 어렵지만요 ). 이 정보를 바탕으로, `추정 소득`이 높은 처치 효과 이질성을 가진 세그먼트를 생성하는 이유를 설명할 수 있습니다.
 
`추정 소득`이 전환율을 높게 예측하기 때문에, 다른 `추정 소득`을 가진 고객들은 S 형태의 전환 곡선에서 서로 다른 위치에 있다고 말할 수 있습니다. 매우 높거나 매우 낮은 `추정 소득`을 가진 고객들은 곡선의 극단에 위치하며, 여기에서 미분이 낮아 전환 증가가 어렵다는 것을 의미하고, 이는 차례로 처치 효과가 작을 가능성이 높다는 것을 의미합니다. 반면, 소득이 중간 범위에 있는 고객들은 전환 곡선의 중간에 위치하며, 여기서 미분이 더 높고, 따라서 처치 효과도 더 높을 가능성이 있습니다. 이것은 '가능성이 높다'고 말하는 것이며, 이론적으로는 어떤 변수가 전환 곡석을 따라가면서 나타나는 미분의 변화를 큰 영향을 줄 만큼 강력한 효과 조절 능력을 가질 수 있지만, 적어도 제 경험상 S형태의 전환 곡선 곡률은 다른 모든 효과 조절에 큰 영향을 주는 경향이 있습니다.

이것은 제 개인적인 견해만은 아닙니다. 여기 Susan Athey가 Columbia Data Science Institute를 위해 한 프레젠테이션에서 가져온 슬라이드가 있습니다. 여기서 그녀는 학생들이 대학 비용을 지불하기 위해 연방 재정 지원을 신청하도록 유도하는 효과에 대해 논의하고 있습니다. 이것도 전환 문제입니다. 그녀가 발견한 것은 이미 전환할 가능성이 높은 학생들을 대상으로 하는 것이 최선의 전략이라는 것입니다. 또한 그녀는 전환 가능성이 낮은 이들을 대상으로 하는 것은 좋지 않은 아이디어라고 말합니다.

![image.png](data/img/hte-binary-outcome/slide-susan-athey.png)
 
잠깐만요! 이것은 당신이 처음에 말한 것과 다릅니다! 당신은 매우 높고 낮은 전환은 모두 낮은 미분값을 가지므로, 낮은 처치 효과를 가진다고 말했습니다!
 
맞습니다. 그러나 실제로, 전환은 S 형태의 전체 곡선을 포괄하지 않습니다. 우리가 일반적으로 가지고 있는 것은 모든 사람이 곡선의 한쪽 끝이나 다른 쪽 끝에 몰려 있는 것입니다. 비즈니스 용어로, 평균 전환율은 거의 50%가 아닙니다. 대부분의 경우, 그것은 70에서 90% 또는 1에서 20%와 같은 것입니다. 이러한 더 현실적인 상황에서, 높은 기준선을 가진 사람들을 대상으로 하는 것은 좋은 아이디어일 수도 있고 나쁜 아이디어일 수도 있습니다.

제가 말하는 것은 이전과 동일한 잠재적 결과를 가지고, 이번에는 전환율이 평균적으로 낮은 상황을 생성하기 위해 `잠재적 결과 > 2`인 경우로 설정합시다. 그 다음, `잠재적 결과 > -2`인 경우로 설정하여 전환율이 높은 상황을 설정합시다.

In [ ]:
df["conversion_low"] = conversion = (latent_outcome > 2).astype(int)
df["conversion_high"] = conversion = (latent_outcome > -2).astype(int)

print("Avg. Low Conversion: ", df["conversion_low"].mean())
print("Avg. High Conversion: ", df["conversion_high"].mean())

전환의 비선형성에 대해 알고 있는 것을 바탕으로, 우리는 이미 무슨 일이 일어날지 예측할 수 있습니다. 낮은 전환 상황에서는 높은 기본 전환율(높은 `추정 소득`)을 가진 사람들을 대상으로 하는 것이 훨씬 더 효과적일 것입니다. 그 이유는 우리가 S 형태의 전환 곡선의 왼쪽에 있기 때문이며, 여기에서는 기본 전환율이 낮을수록 미분값이 작아집니다. 이 지역에서는 **높은 기본 전환율이 높은 처치 효과로 전환됩니다.** 따라서, 우리는 높은 기본 전환율을 가진 사람들, 즉 높은 `추정 소득`을 가진 사람들에게 유도 수단을 적용해야 합니다.

In [ ]:
cumulative_effect_fn = cumulative_effect_curve(df, "nudge", "conversion_low", min_rows=500)

age_cumm_effect_latent = cumulative_effect_fn(prediction="age")
inc_cumm_effect_latent = cumulative_effect_fn(prediction="estimated_income")

plt.plot(age_cumm_effect_latent, label="age")
plt.plot(inc_cumm_effect_latent, label="est. income")
plt.xlabel("Percentile")
plt.ylabel("Effect on Conversino");
plt.legend();

우리가 예측했듯이, 높은 `추정 소득`을 가진 사람들, 즉 높은 기본 전환율을 가진 사람들이 훨씬 더 높은 처치 효과를 보입니다.

이제 전환율이 평균적으로 높은 다른 상황에서는, **높은 기본 전환율을 가진 사람들이 더 낮은 처치 효과를 가질 것입니다.** 따라서 높은 `추정 소득`을 가진 사람들을 대상으로 하는 것은 나쁜 아이디어입니다. 이는 높은 `추정 소득`을 가진 사람들이 더 낮은 처치 효과를 가진다고 보여주는 반전된 누적 효과 곡선을 통해 확인할 수 있습니다.

In [ ]:
cumulative_effect_fn = cumulative_effect_curve(df, "nudge", "conversion_high", min_rows=500)

age_cumm_effect_latent = cumulative_effect_fn(prediction="age")
inc_cumm_effect_latent = cumulative_effect_fn(prediction="estimated_income")

plt.plot(age_cumm_effect_latent, label="age")
plt.plot(inc_cumm_effect_latent, label="est. income")
plt.xlabel("Percentile")
plt.ylabel("Effect on Conversino")
plt.legend();

결과가 이진일 때 처리 효과는 로지스틱 함수의 곡률 (도함수)에 주로 의존합니다. 이는 이진 형태의 결과를 갖는 경우, 그 결과에 대한 처리 효과가 로지스틱 함수의 형태나 변화 정도에 큰 영향을 받는다는 것을 의미합니다.

![image.png](data/img/hte-binary-outcome/logistic.png)
 
예를 들어, 전환 문제에서 **평균 전환율이 낮으면** 우리는 로지스틱 곡선의 왼쪽에 있게 되고, **높은 기본 전환율에서 처리 효과가 더 클 것입니다.** 이는 이미 전환 가능성이 높은 고객을 대상으로 한 개입(유도) 정책으로 해석될 수 있습니다. 반면, **평균 전환율이 높으면** 로지스틱 곡선의 오른쪽에 위치하게 되며, 여기서는 **기본 전환율이 낮은 고객에 대한 처리 효과(도함수)가 더 클 것입니다.**
 
이 내용을 간단히 요약하면: **기본 전환율이 50%에 가까운 대상을 처리하면 됩니다.** 여기엔 탄탄한 수학적 논리가 있습니다: 로지스틱 함수의 도함수는 50%에서 최고점에 도달하므로, 그 지점에 가까운 대상에 초점을 맞추는 것이 좋습니다.
 
이 경우에는 흔치 않게 상식과 수학이 일치하는 좋은 예입니다. 마케팅 분야에서 이런 전환 문제가 매우 흔한데, 여기서는 매우 낮은 전환 확률을 가진 대상(잃을 것 같은 베팅)이나 매우 높은 전환 확률을 가진 대상(확실한 승리)을 목표로 삼지 말고, 중간 범위에 있는 대상을 목표로 해야 한다는 믿음이 있습니다. 이것은 공식적인 인과 논리를 사용해 우리가 알아낸 것과 정확히 같은 내용이라는 점에서 매우 흥미롭습니다.

# Continues Treatment and Non Linearity
 
우리는 이진 결과가 이질적인 처리 효과 분석을 어렵게 만드는 예시를 심층적으로 탐구했습니다. 그러나 이 현상은 마케팅의 전환 문제에 국한되지 않습니다. 예를 들어, 2021년에 세계는 첫 COVID-19 백신을 일반 대중에게 제공했습니다. 당시 주요 질문은 누가 먼저 백신을 맞아야 할까였습니다. 이는 이질적 처리 효과 문제입니다. 정책 입안자들은 가장 이득을 볼 사람들을 먼저 백신 접종하고자 했습니다. 이 상황에서 처리 효과는 사망이나 입원 방지입니다. 그렇다면 백신 접종으로 사망이나 입원이 가장 감소한 사람들은 누구일까요? 대부분의 국가에서는 노인과 기존 건강 상태가 좋지 않은 사람들이었습니다. 이제 이 분들은 **COVID-19에 감염될 경우 사망할 가능성이 더 높은 사람들입니다.** 또한 COVID-19의 사망률은 다행히 50%보다 훨씬 낮아 로지스틱 함수의 왼쪽에 위치합니다. 이 지역에서는 마케팅에 대해 한 논리와 마찬가지로, COVID-19에 감염될 경우 높은 기본 사망 확률을 가진 사람들을 치료하는 것이 합리적입니다. 이는 앞서 언급한 그룹에 해당합니다. 이것이 우연일까요? 아마도. 저는 건강 전문가가 아니므로 여기서 잘못된 점이 있을 수 있습니다. 하지만 이 논리는 매우 타당해 보입니다.

마케팅 유도와 COVID-19 백신의 경우, **처리 효과 이질성의 주요 복잡화 요인은 결과 함수 $Y(0)$의 비선형성**입니다. 이 비선형성은 $Y(0)$ 에서 $Y(1)$으로 가면서 결과의 증가가 주로 결과 함수의 곡률 때문이라는 것을 의미합니다. 이진 결과에서 $E[Y|X]$가 로지스틱 형태를 따르는 것을 보았습니다. 하지만 이 문제는 더 일반적입니다. 특히 처리가 연속 변수인 경우 비즈니스에서 자주 발생합니다. 이 아이디어를 더 명확하게 하기 위해 마지막 예를 살펴보는 것이 좋겠습니다.

클래식한 가격 책정 문제를 고려해 봅시다. 넷플릭스나 HBO 같은 스트리밍 회사에서 일한다고 가정합니다. 회사가 원하는 핵심 질문은 고객에게 어떤 가격을 부과할 것인가입니다. 이를 위해 그들은 5 BRL/월, 10 BRL/월, 15 BRL/월, 20 BRL/월로 다양한 가격의 거래를 무작위로 고객에게 할당하는 실험을 실시합니다. 이를 통해 고객들이 가격 인상에 얼마나 민감한지, 또 어떤 유형의 고객이 더 민감한지를 알아보려고 합니다. 아래 그래프에서는 이 실험의 결과를 두 고객 세그먼트로 나누어 볼 수 있습니다: `A`, 추정 소득이 높은 고객들과 `B`, 추정 소득이 낮은 고객들입니다.

In [ ]:
data = pd.DataFrame(dict(
    segment= ["b", "b", "b", "b",  "a", "a", "a", "a",],
    price=[5, 10, 15, 20, ] * 2,
    sales=[5100, 5000, 4500, 3000,  5350, 5300, 5000, 4500]
))

plt.figure(figsize=(8,4))
sns.lineplot(data=data, x="price", y="sales", hue="segment")
plt.title("Avg. Sales by Price (%) by Customer Segment");

이 데이터를 통해 회사는 다음과 같은 질문에 대한 답을 찾고자 합니다: 할인에 더 민감한 고객은 누구인가? 다시 말해, **가격에 대한 민감도(판매의 가격 탄력성)별로 고객을 어떻게 순위지을 수 있을까**? 곡선을 보면 세그먼트 `A`가 할인에 대해 전반적으로 덜 민감하다는 느낌을 받지만, 더 많은 수익을 생성한다는 것도 볼 수 있습니다. 하지만 곡선에 어느 정도 곡률이 있다는 것도 볼 수 있습니다. 실제로, 이 곡률을 고려하면 `A`와 `B` 고객 간의 처리 효과 순위는 더 이상 명확하지 않습니다. 처리 효과는 그들이 처리 곡선에서 어디에 있는지에도 달려 있습니다. 예를 들어, 15 BRL에서 10 BRL로 갈 때 세그먼트 `A` 고객에 대한 처리 효과는 5 BRL에서 10 BRL로 갈 때 세그먼트 B 고객에 대한 처리 효과보다 높습니다.
`B`:

$$
E[Y(10) - Y(5) | Seg=B] < E[Y(15) - Y(10) | Seg=A]
$$

이 실험에 대한 처리 효과의 결과를 순서대로 정리한다면, 다음과 같은 모습일 것입니다:

In [ ]:
plt.figure(figsize=(8,4))
sns.lineplot(data=data, x="price", y="sales", hue="segment")

plt.annotate("1", (8, 5350), bbox=dict(boxstyle="round", fc="1"))
plt.annotate("2", (8, 5000), bbox=dict(boxstyle="round", fc="1"))
plt.annotate("3", (13, 5100), bbox=dict(boxstyle="round", fc="1"))
plt.annotate("4", (13, 4700), bbox=dict(boxstyle="round", fc="1"))
plt.annotate("4", (17, 4800), bbox=dict(boxstyle="round", fc="1"))
plt.annotate("5", (17, 3900), bbox=dict(boxstyle="round", fc="1"))

plt.title("Ordering of the Effect of Increasing Price");

이 예시에서도 결과가 이진일 때와 마찬가지로, **처리 효과는 결과와 상관관계를 가집니다.** 판매가 높을수록(가격이 낮을수록) 절대 처리 효과는 낮아지고, 판매가 낮을수록(가격이 높을수록) 절대 처리 효과도 낮아집니다. 하지만 이 경우는 **처리 효과가 결과뿐만 아니라 처리 수준과도 상관관계를 가지기 때문에 더 복잡합니다.** 이로 인해 반사실적 질문에 답하기가 더 까다로워집니다. 예를 들어, 실험 데이터가 다음 그림과 같이 `A` 세그먼트(부유한 인구)에는 높은 가격을 테스트하지만 `A` 인구에는 낮은 가격만 테스트하는 것처럼 상상해 보세요. 이는 회사들이 보다 합리적이라고 생각하는 처리를 중심으로 실험하는 경우가 많기 때문에 흔한 일입니다.

In [ ]:
data = pd.DataFrame(dict(
    segment= ["b", "b", "b", "b",  "a", "a", "a", "a",],
    price=[5, 10, 15, 20, ] * 2,
    sales=[5100, 5000, 4500, 3000,  5350, 5300, 5000, 4500]
))

plt.figure(figsize=(8,4))
sns.lineplot(data=data.loc[lambda d: (d["segment"] == "a") | (d["price"] < 12) ], x="price", y="sales", hue="segment")
plt.title("Avg. Sales by Price (%) by Customer Segment");

처리 효과의 결과를 단순히 집계하면 세그먼트 `A`가 세그먼트 `B`보다 가격 인상에 대해 훨씬 더 탄력적인 것처럼 보일 것입니다(절대 처리 효과가 더 큼). 그러나 이는 세그먼트 `B`에 대해 낮은 처리 효과 지역을 탐색했기 때문에 그렇게 보이는 것입니다.
 
처리 효과가 처리와 결과의 위치에 따라 변할 때 어떻게 해야 할까요? 솔직히 말해서, 이것은 여전히 활발한 연구 분야입니다. 실질적으로 할 수 있는 최선의 방법은 어떤 유형의 고객이 처리에 더 민감한지를 답하려고 할 때 **매우 신중해야 합니다.** 비교하는 고객 유형이 모두 같은 처리 분포를 가졌는지 확인하세요. 그렇지 않다면, 처리 효과를 외삽하는 데 매우 회의적이어야 합니다. 예를 들어, 위의 예에서 `B` 고객이 가격 인상에 덜 민감해 보이지만, 이 세그먼트에 10BRL을 넘는 높은 가격을 책정한다면 여전히 그럴지는 알 수 없습니다.

처리와 결과 사이의 관계를 선형화하는 것도 시도해 볼 수 있습니다. 여기서의 아이디어는 처리나 결과(또는 둘 다)를 변환하여 곡률을 없애고 그들 사이의 관계를 선형처럼 보이게 하는 것입니다. 선은 일정한 도함수를 가지므로, 이는 곡선에서 위치에 따라 변하는 처리 효과 문제를 해결할 수 있습니다. 예를 들어, 가격 변수를 부정적으로 만들고 4로 지수화한 후 부호를 바꾸면 어느 정도 선형적인 관계를 얻을 수 있습니다. 이 변환된 데이터에서는 `A`가 `B`보다 가격 인상에 덜 민감하다는 것이 훨씬 더 의미가 있으며, 이제는 곡선에서의 위치에 의존하지 않습니다.

In [ ]:

plt.figure(figsize=(8,4))
sns.lineplot(data=data.assign(price = lambda d: -1*(-d["price"]**4)),
             x="price", y="sales", hue="segment")
plt.title("Avg. Sales by -(-price^4)");

이 접근법에는 여러 단점이 있습니다. 첫째, 곡선을 선형화하는 것이 항상 가능한 것은 아닙니다. 예제에서도 이 선형화가 완벽하지 않음을 명확히 볼 수 있습니다. 하지만 더 중요한 것은, 때때로 곡률을 무시하는 것이 비즈니스 상 의미가 없다는 것입니다. 가격 설정 예제에서, 15의 가격에 대해 `A` 고객을 5의 가격에 대한 `B` 고객보다 더 민감하게 처리하는 것이 타당할 수 있습니다. 이는 `A` 고객의 가격을 15에서 10으로 낮추는 합리적인 결정으로 이어질 수 있지만, `B` 고객의 가격에는 아무런 조치를 취하지 않을 수도 있습니다.

## Key Concepts
 
제가 더 많은 질문을 제기했을지 모르겠지만, 때때로 문제에 대해 할 수 있는 최선은 그 문제를 충분히 인식하는 것입니다. 이 장에서, 비선형 결과에 대해 신경 써야 할 때 발생하는 복잡함에 대해 여러분의 눈을 뜨게 하는 데 성공했기를 바랍니다.

이진 결과에 관한 문제는 일반적이고 더 많이 연구된 문제입니다. 이 경우, 처리 효과는 평균 결과가 0.5에 가까울수록 높아지는 경향이 있습니다. 결과가 0과 1로 제한되어 있기 때문에, 0이나 1에 너무 가까우면 효과는 매우 작아질 수 있습니다.

연속적 결과 상황에서 비선형성이 발생하면 상황이 더 복잡해집니다. 여기서는 문제에 대해 신중하게 생각하는 것이 최선입니다. 처리 기준을 떠나서 처리 효과에 더 관심이 있는지, 아니면 기준이 중요한지에 대한 답을 시도해 보세요. 이것만으로도 중요한 지침 원칙이 될 것입니다.

## Reference
 
이 글에 쓰인 대부분은 이 문제와 관련된 저자의 경험에서 비롯되었습니다. 하지만, 이 주제와 관련된 학술 논문도 하나 찾았습니다: *Causal Classification: Treatment Effect Estimation vs. Outcome Prediction* 이라는 제목의 Fernández-Loría와 Provost의 논문은 처리 효과가 결과 변수와 상관관계를 가질 때의 경우에 대해 다룹니다.

## Contribute
 
Causal Inference for the Brave and True는 인과추론,통계학에 대한 오픈소스 자료입니다.이 자료는 금전적으로나 모든 분들이 쉽게 접근하실 수 있도록 하는 것이 목표입니다. 또한, 이 책은 Python 기반의 무료 소프트웨어만 사용해요. 여러분들께서 이 자료가 가치 있다고 생각하시고, 금전적으로 지원을 원하신다면 [Patreon](https://www.patreon.com/causal_inference_for_the_brave_and_true)를 방문해주세요. 만약 여러분이 금전적으로 기여하기가 쉽지 않으시다면, 오타수정, 수정 제안, 이해하기 난해한 부분에 대한 피드백 제공등을 통해 도움을 주실 수 있어요. 이 책의 Github 저장소  [이슈 페이지](https://github.com/matheusfacure/python-causality-handbook/issues)를 방문해주세요. 마지막으로 이 자료가 여러분의 마음에 드셨다면 도움이 될 수 있는 다른 사람들과 공유해주시고,[한국어 번역 자료](https://github.com/CausalInferenceLab/Causal-Inference-with-Python/stargazers)와 [해당 번역본의 원서](https://github.com/matheusfacure/python-causality-handbook/stargazers)에 star 부탁드립니다!